# This notebook has all of the steps needed to configure your workspace and download imagery for processing and analysis

## import all necessary packages

In [1]:
import os
import numpy as np
from src.landsatUtil.landsat.downloader import Downloader
from src.utils.data_directory_manager import DataDirectoryManager
from src.utils.image_correction import LandsatTOACorrecter
from src.models.antarctic_rock_outcrop_os import OutcropLabeler

## Configure project directory

In [2]:
project_directory = "/home/dsa/DSA/qc"
dm = DataDirectoryManager(project_directory)

2019-11-14 13:10:43,654 - scene_downloader_log
INFO
Data directory created at /home/dsa/DSA/qc/raw

2019-11-14 13:10:43,657 - scene_downloader_log
INFO
Data directory created at /home/dsa/DSA/qc/corrected

2019-11-14 13:10:43,660 - scene_downloader_log
INFO
Data directory created at /home/dsa/DSA/qc/labels

2019-11-14 13:10:43,663 - scene_downloader_log
INFO
Data directory created at /home/dsa/DSA/qc/downloads

2019-11-14 13:10:43,665 - scene_downloader_log
INFO
Data directory created at /home/dsa/DSA/qc/coastline



## Load list of scene ids to be manipulated
### This cell should create a list of scene ids from any source file that can be used to download scenes, correct them, label them, etc

In [5]:
# This particular example loads a list from a file where each line is a single scene id
scene_file_path = os.path.join(dm.project_dir, "qc_ids.txt")
with open(scene_file_path, 'r') as scene_file:
    scene_ids = [i[:-1] for i in scene_file]

## Download scenes by id
### specific bands can be downloaded by introducing a second parameter
### It is strongly recommended that you specify bands because the provided scenes come as .tifs instead of all bands provided as a .tar.bz file that must be extracted (this extraction takes a very long time)

Landsat band information can be found here: https://www.usgs.gov/land-resources/nli/landsat/landsat-8?qt-science_support_page_related_con=0#qt-science_support_page_related_con

In [8]:
bands = [1]
# specify dm.download_dir if not providing a band parameter
downloader = Downloader(download_dir = os.path.join(dm.project_dir, "full_scene_ref_bands")) 
downloader.download(scene_ids, bands=bands)

===> Source: AWS S3
===> Downloading: LC80311222014338LGN00_B1.TIF


     0%     12.6 KiB      12.6 KiB/s            1:12:59 ETA    
     2%      1.1 MiB     772.7 KiB/s            0:01:09 ETA    
    15%      8.5 MiB       4.2 MiB/s            0:00:10 ETA    
    30%     16.3 MiB       6.5 MiB/s            0:00:05 ETA    
    44%     23.9 MiB       7.9 MiB/s            0:00:03 ETA    
    56%     30.5 MiB       8.7 MiB/s            0:00:02 ETA    
    71%     38.7 MiB       9.6 MiB/s            0:00:01 ETA    
    86%     46.6 MiB      10.3 MiB/s            0:00:00 ETA    



     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_BQA.TIF


     0%     12.6 KiB      12.6 KiB/s            0:03:57 ETA    
    38%      1.1 MiB     762.9 KiB/s            0:00:02 ETA    



     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_MTL.txt


...

     0%     12.6 KiB      12.6 KiB/s            0:03:44 ETA    
    40%      1.1 MiB     733.9 KiB/s            0:00:02 ETA    



     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_MTL.txt


   100%      7.4 KiB       7.4 KiB/s            0:00:00 ETA    



     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_BQA.TIF
     LC81681112014002LGN00_BQA.TIF already exists on your system
     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_BQA.TIF
     LC81681112014002LGN00_BQA.TIF already exists on your system
     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_BQA.TIF
     LC81681112014002LGN00_BQA.TIF already exists on your system
     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_BQA.TIF
     LC81681112014002LGN00_BQA.TIF already exists on your system
     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00
===> Downloading: LC81681112014002LGN00_BQA.TIF
     LC81681112014002LGN00_BQA.TIF already exists on your system
     stored at /home/dsa/DSA/qc/full_scene_ref_bands/LC81681

['/home/dsa/DSA/qc/full_scene_ref_bands/LC80311222014338LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC82091172014001LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC82201072015017LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC81431082013352LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC82071182013336LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC80631112014002LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC82131132013362LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC81041072013303LGN00',
 '/home/dsa/DSA/qc/full_scene_ref_bands/LC81681112014002LGN00']

# Process downloaded scenes
## only run this cell if downloading .tar.bz files into dm.download_dir

In [ ]:
dm.untar_scenes(scene_ids)

# Correct scenes

In [9]:
for i in scene_ids:
    corrector = LandsatTOACorrecter(i)
    corrector.correct_toa_brightness_tem(dm.corrected_image_dir)
    corrector.correct_toa_reflectance(dm.corrected_image_dir)

TypeError: __init__() missing 1 required positional argument: 'scene_path'

# Rasterize vector labels

# stack bands and split into chunks along with labels